In [715]:
# General Tools
import pandas as pd
import numpy as np
from pandas.tools import plotting
import matplotlib.pyplot as plt

# for data preparation
from sklearn.model_selection import train_test_split

# for ML Classification algorithm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

#for model evaluation
from sklearn.metrics import confusion_matrix, accuracy_score

In [716]:
%%markdown
## Load Dataset Train and Test

## Load Dataset Train and Test

In [717]:
raw_train = pd.read_csv("train.csv", index_col=0)
raw_train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [718]:
raw_test = pd.read_csv("test.csv", index_col=0)
raw_test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [719]:
print(raw_train.shape)
print(raw_test.shape)

(891, 11)
(418, 10)


In [720]:
%%markdown
## Drop Unused Column 

## Drop Unused Column 

In [721]:
train_data = raw_train.drop(['Cabin', 'Name', 'Ticket'], axis=1)
train_data.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S
2,1,1,female,38.0,1,0,71.2833,C
3,1,3,female,26.0,0,0,7.9250,S


In [722]:
test_data = raw_test.drop(['Cabin', 'Name', 'Ticket'], axis=1)
test_data.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
892,3,male,34.5,0,0,7.8292,Q
893,3,female,47.0,1,0,7.0000,S
894,2,male,62.0,0,0,9.6875,Q


In [723]:
%%markdown
## Handling Missing Value

## Handling Missing Value

In [724]:
print(train_data.isnull().sum())

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64


In [725]:
print(test_data.isnull().sum())

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64


In [726]:
%%markdown
### For Data Train

### For Data Train

In [727]:
Age_train_mean_m = np.mean(train_data[train_data.Sex == 'male'])['Age']
Age_train_mean_f = np.mean(train_data[train_data.Sex == 'female'])['Age']

In [728]:
Embarked_train_mode = train_data.Embarked.mode()[0]

In [729]:
# Fill NA
train_data.loc[train_data.Sex == 'male', ['Age']] = train_data.loc[train_data.Sex == 'male', ['Age']].fillna(Age_train_mean_m)
train_data.loc[train_data.Sex == 'female', ['Age']] = train_data.loc[train_data.Sex == 'female', ['Age']].fillna(Age_train_mean_f)
train_data['Embarked'] = train_data.Embarked.fillna(Embarked_train_mode)

In [730]:
%%markdown
### For Data Test

### For Data Test

In [731]:
Age_test_mean_m = np.mean(test_data[test_data.Sex == 'male'])['Age']
Age_test_mean_f = np.mean(test_data[test_data.Sex == 'female'])['Age']

In [732]:
Fare_test_mean = np.mean(test_data.Fare)

In [733]:
# Fill NA
test_data.loc[test_data.Sex == 'male', ['Age']] = test_data.loc[test_data.Sex == 'male', ['Age']].fillna(Age_test_mean_m)
test_data.loc[test_data.Sex == 'female', ['Age']] = test_data.loc[test_data.Sex == 'female', ['Age']].fillna(Age_test_mean_f)
test_data['Fare'] = test_data.Fare.fillna(Fare_test_mean)

In [734]:
# check NA again
print(train_data.isnull().sum())
print(test_data.isnull().sum())

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64


In [735]:
%%markdown
## Creating Data Dummies

## Creating Data Dummies

In [736]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    891 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 62.6+ KB


In [737]:
train_num = pd.get_dummies(train_data, columns=['Sex', 'Embarked', 'Pclass'], drop_first=True)
test_num = pd.get_dummies(test_data, columns=['Sex', 'Embarked', 'Pclass'], drop_first=True)

In [738]:
train_num.head()

,Survived,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
PassengerId,,,,,,,,,,
1,0,22.0,1,0,7.2500,1,0,1,0,1
2,1,38.0,1,0,71.2833,0,0,0,0,0
3,1,26.0,0,0,7.9250,0,0,1,0,1
4,1,35.0,1,0,53.1000,0,0,1,0,0
5,0,35.0,0,0,8.0500,1,0,1,0,1


In [739]:
%%markdown
## Feature Engineering

## Feature Engineering

In [740]:
train_num['Family'] = train_num['SibSp'] + train_num['Parch']
test_num['Family'] = test_num['SibSp'] + test_num['Parch']

In [741]:
#drop
train_num = train_num.drop(['SibSp', 'Parch'], axis=1)
test_num = test_num.drop(['SibSp', 'Parch'], axis=1)

In [742]:
%%markdown
## Train Test Split

## Train Test Split

In [743]:
X=train_num.drop(['Survived'],1)
Y=train_num['Survived']
train_X,test_X,train_Y,test_Y =train_test_split(X,Y,test_size=0.2, random_state=17)

In [744]:
%%markdown
## Logistic Regression

## Logistic Regression

In [745]:
logreg = LogisticRegression(random_state = 17)
logreg.fit(train_X, train_Y)
logregpred_train = logreg.predict(train_X)
logregpred_test = logreg.predict(test_X)

In [746]:
print("Predict Data Training")
cm_train = confusion_matrix(train_Y,logregpred_train)

total_train=sum(sum(cm_train))
accuracy_train = (cm_train[0,0]+cm_train[1,1])/total_train
sensitivity_train = cm_train[1,1]/(cm_train[1,0]+cm_train[1,1])
specificity_train = cm_train[0,0]/(cm_train[0,0]+cm_train[0,1])

print('Confusion Matrix : \n', cm_train)
print('Accuracy : ', accuracy_train)
print('Sensitivity : ', sensitivity_train)
print('Specificity : ', specificity_train)

Predict Data Training
Confusion Matrix : 
 [[398  45]
 [ 86 183]]
Accuracy :  0.8160112359550562
Sensitivity :  0.6802973977695167
Specificity :  0.8984198645598194


In [747]:
print("Predict Data Validation")
cm_test = confusion_matrix(test_Y,logregpred_test)

total_test=sum(sum(cm_test))
accuracy_test = (cm_test[0,0]+cm_test[1,1])/total_test
sensitivity_test = cm_test[1,1]/(cm_test[1,0]+cm_test[1,1])
specificity_test = cm_test[0,0]/(cm_test[0,0]+cm_test[0,1])

print('Confusion Matrix : \n', cm_test)
print('Accuracy : ', accuracy_test)
print('Sensitivity : ', sensitivity_test)
print('Specificity : ', specificity_test)

Predict Data Validation
Confusion Matrix : 
 [[91 15]
 [24 49]]
Accuracy :  0.7821229050279329
Sensitivity :  0.6712328767123288
Specificity :  0.8584905660377359


In [748]:
%%markdown
## Decision Tree

## Decision Tree

In [749]:
dtree = DecisionTreeClassifier(min_samples_split=400, random_state = 17)
dtree.fit(train_X, train_Y)
dtree_train = dtree.predict(train_X)
dtree_test = dtree.predict(test_X)

In [750]:
print("Predict Data Training")
cm_train = confusion_matrix(train_Y,dtree_train)

total_train=sum(sum(cm_train))
accuracy_train = (cm_train[0,0]+cm_train[1,1])/total_train
sensitivity_train = cm_train[1,1]/(cm_train[1,0]+cm_train[1,1])
specificity_train = cm_train[0,0]/(cm_train[0,0]+cm_train[0,1])

print('Confusion Matrix : \n', cm_train)
print('Accuracy : ', accuracy_train)
print('Sensitivity : ', sensitivity_train)
print('Specificity : ', specificity_train)

Predict Data Training
Confusion Matrix : 
 [[373  70]
 [ 73 196]]
Accuracy :  0.7991573033707865
Sensitivity :  0.7286245353159851
Specificity :  0.8419864559819413


In [751]:
print("Predict Data Validation")
cm_test = confusion_matrix(test_Y,dtree_test)

total_test=sum(sum(cm_test))
accuracy_test = (cm_test[0,0]+cm_test[1,1])/total_test
sensitivity_test = cm_test[1,1]/(cm_test[1,0]+cm_test[1,1])
specificity_test = cm_test[0,0]/(cm_test[0,0]+cm_test[0,1])

print('Confusion Matrix : \n', cm_test)
print('Accuracy : ', accuracy_test)
print('Sensitivity : ', sensitivity_test)
print('Specificity : ', specificity_test)

Predict Data Validation
Confusion Matrix : 
 [[87 19]
 [20 53]]
Accuracy :  0.7821229050279329
Sensitivity :  0.726027397260274
Specificity :  0.8207547169811321


In [752]:
%%markdown
## SVM

## SVM

In [753]:
svm = SVC()
svm.fit(train_X, train_Y) 
svm_train = svm.predict(train_X)
svm_test = svm.predict(test_X)

In [754]:
print("Predict Data Training")
cm_train = confusion_matrix(train_Y,svm_train)

total_train=sum(sum(cm_train))
accuracy_train = (cm_train[0,0]+cm_train[1,1])/total_train
sensitivity_train = cm_train[1,1]/(cm_train[1,0]+cm_train[1,1])
specificity_train = cm_train[0,0]/(cm_train[0,0]+cm_train[0,1])

print('Confusion Matrix : \n', cm_train)
print('Accuracy : ', accuracy_train)
print('Sensitivity : ', sensitivity_train)
print('Specificity : ', specificity_train)

Predict Data Training
Confusion Matrix : 
 [[423  20]
 [ 54 215]]
Accuracy :  0.8960674157303371
Sensitivity :  0.7992565055762082
Specificity :  0.9548532731376975


In [755]:
print("Predict Data Validation")
cm_test = confusion_matrix(test_Y,svm_test)

total_test=sum(sum(cm_test))
accuracy_test = (cm_test[0,0]+cm_test[1,1])/total_test
sensitivity_test = cm_test[1,1]/(cm_test[1,0]+cm_test[1,1])
specificity_test = cm_test[0,0]/(cm_test[0,0]+cm_test[0,1])

print('Confusion Matrix : \n', cm_test)
print('Accuracy : ', accuracy_test)
print('Sensitivity : ', sensitivity_test)
print('Specificity : ', specificity_test)

Predict Data Validation
Confusion Matrix : 
 [[95 11]
 [40 33]]
Accuracy :  0.7150837988826816
Sensitivity :  0.4520547945205479
Specificity :  0.8962264150943396


In [756]:
%%markdown
## KNN

## KNN

In [757]:
knn = KNeighborsClassifier()
knn.fit(train_X, train_Y) 
knn_train = knn.predict(train_X)
knn_test = knn.predict(test_X)

In [758]:
print("Predict Data Training")
cm_train = confusion_matrix(train_Y,knn_train)

total_train=sum(sum(cm_train))
accuracy_train = (cm_train[0,0]+cm_train[1,1])/total_train
sensitivity_train = cm_train[1,1]/(cm_train[1,0]+cm_train[1,1])
specificity_train = cm_train[0,0]/(cm_train[0,0]+cm_train[0,1])

print('Confusion Matrix : \n', cm_train)
print('Accuracy : ', accuracy_train)
print('Sensitivity : ', sensitivity_train)
print('Specificity : ', specificity_train)

Predict Data Training
Confusion Matrix : 
 [[385  58]
 [ 82 187]]
Accuracy :  0.8033707865168539
Sensitivity :  0.6951672862453532
Specificity :  0.8690744920993227


In [759]:
print("Predict Data Validation")
cm_test = confusion_matrix(test_Y,knn_test)

total_test=sum(sum(cm_test))
accuracy_test = (cm_test[0,0]+cm_test[1,1])/total_test
sensitivity_test = cm_test[1,1]/(cm_test[1,0]+cm_test[1,1])
specificity_test = cm_test[0,0]/(cm_test[0,0]+cm_test[0,1])

print('Confusion Matrix : \n', cm_test)
print('Accuracy : ', accuracy_test)
print('Sensitivity : ', sensitivity_test)
print('Specificity : ', specificity_test)

Predict Data Validation
Confusion Matrix : 
 [[92 14]
 [37 36]]
Accuracy :  0.7150837988826816
Sensitivity :  0.4931506849315068
Specificity :  0.8679245283018868


In [760]:
%%markdown
## Random Forest

## Random Forest

In [853]:
rf = RandomForestClassifier(n_estimators=17, random_state=17, min_samples_leaf=6)
rf.fit(train_X, train_Y) 
rf_train = rf.predict(train_X)
rf_test = rf.predict(test_X)

In [854]:
print("Predict Data Training")
cm_train = confusion_matrix(train_Y,rf_train)

total_train=sum(sum(cm_train))
accuracy_train = (cm_train[0,0]+cm_train[1,1])/total_train
sensitivity_train = cm_train[1,1]/(cm_train[1,0]+cm_train[1,1])
specificity_train = cm_train[0,0]/(cm_train[0,0]+cm_train[0,1])

print('Confusion Matrix : \n', cm_train)
print('Accuracy : ', accuracy_train)
print('Sensitivity : ', sensitivity_train)
print('Specificity : ', specificity_train)

Predict Data Training
Confusion Matrix : 
 [[423  20]
 [ 72 197]]
Accuracy :  0.8707865168539326
Sensitivity :  0.7323420074349443
Specificity :  0.9548532731376975


In [855]:
print("Predict Data Validation")
cm_test = confusion_matrix(test_Y,rf_test)

total_test=sum(sum(cm_test))
accuracy_test = (cm_test[0,0]+cm_test[1,1])/total_test
sensitivity_test = cm_test[1,1]/(cm_test[1,0]+cm_test[1,1])
specificity_test = cm_test[0,0]/(cm_test[0,0]+cm_test[0,1])

print('Confusion Matrix : \n', cm_test)
print('Accuracy : ', accuracy_test)
print('Sensitivity : ', sensitivity_test)
print('Specificity : ', specificity_test)

Predict Data Validation
Confusion Matrix : 
 [[95 11]
 [21 52]]
Accuracy :  0.8212290502793296
Sensitivity :  0.7123287671232876
Specificity :  0.8962264150943396


In [786]:
%%markdown
## Submission to Kaggle

## Submission to Kaggle

In [861]:
rf_testing = rf.predict(test_num)

In [862]:
df_pred = pd.DataFrame({'Survived': rf_testing})

In [863]:
df_submit = pd.concat([test_num.reset_index()['PassengerId'], df_pred], axis=1)

In [864]:
df_submit = df_submit.set_index('PassengerId')

In [865]:
df_submit.to_csv('result1.csv',sep=',',index=True)